In [ ]:
import warnings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
warnings.filterwarnings("ignore")

In [26]:
import glob
import os
warnings.filterwarnings("ignore")
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

def load_pdf_file(data):
    """
    Load all PDF files from the given directory using DirectoryLoader and PyPDFLoader.
    
    Args:
        data (str): Path to the folder containing PDF files.
    
    Returns:
        list: List of langchain Document objects.
    """
    if not os.path.exists(data):
        raise FileNotFoundError(f"Directory not found: '{data}'")
        
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    docs = loader.load()
    return docs


In [27]:
docs = load_pdf_file('.')  # '.' means current directory
print(f"Loaded {len(docs)} documents.")

Ignoring wrong pointing object 1304 0 (offset 0)
Ignoring wrong pointing object 1306 0 (offset 0)
Ignoring wrong pointing object 1458 0 (offset 0)
Ignoring wrong pointing object 1502 0 (offset 0)
Ignoring wrong pointing object 1812 0 (offset 0)
Ignoring wrong pointing object 1815 0 (offset 0)
Ignoring wrong pointing object 3289 0 (offset 0)
Ignoring wrong pointing object 3436 0 (offset 0)
Ignoring wrong pointing object 4032 0 (offset 0)
Ignoring wrong pointing object 4129 0 (offset 0)
Ignoring wrong pointing object 4153 0 (offset 0)
Ignoring wrong pointing object 4185 0 (offset 0)
Ignoring wrong pointing object 4769 0 (offset 0)
Ignoring wrong pointing object 4806 0 (offset 0)
Ignoring wrong pointing object 4832 0 (offset 0)
Ignoring wrong pointing object 4888 0 (offset 0)
Ignoring wrong pointing object 5030 0 (offset 0)
Ignoring wrong pointing object 5104 0 (offset 0)
Ignoring wrong pointing object 5114 0 (offset 0)
Ignoring wrong pointing object 5286 0 (offset 0)
Ignoring wrong point

Loaded 1784 documents.


In [7]:
def text_split(extr_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extr_data)
    return text_chunks

In [8]:
text_chunks = text_split(docs)
print("Length of text chunks: ", len(text_chunks))

Length of text chunks:  11833


In [10]:
# Download the Embedding model from Hugging Face
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = download_hugging_face_embedding()


C:\Users\ENG.YAHYA\AppData\Local\Temp\ipykernel_27116\361975650.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [12]:
#os.chdir("C:\\Users\\ENG.YAHYA\\Desktop\\projects\\FinTutor-Chatbot-Project\\model")

In [13]:
import sentence_transformers
print(sentence_transformers.__version__)

5.0.0


In [50]:
#query_result = embeddings.embed_query("Hello world")
#print(query_result)
#len(query_result)


In [15]:
print(embeddings.model_name)

sentence-transformers/all-MiniLM-L6-v2


In [43]:
from dotenv import load_dotenv
load_dotenv()

True

In [44]:
pinecone_api_key = os.getenv("PINECONE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")



In [21]:
from pinecone.grpc import PineconeGRPC as pinecone
from pinecone import ServerlessSpec
import os 

pc = pinecone(api_key=pinecone_api_key)

index_name = "fintutor"

pc.create_index(
    index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws",region="us-east-1")
)



{
    "name": "fintutor",
    "metric": "cosine",
    "host": "fintutor-bcouli4.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [45]:
import os
pinecone_api_key = os.getenv("PINECONE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")


In [30]:
from langchain_pinecone import PineconeVectorStore
docsearch=PineconeVectorStore.from_documents(
    documents = text_chunks,
    index_name= index_name,
    embedding=embeddings
    )

In [31]:
## Load the Existing index
from langchain_pinecone import PineconeVectorStore
## Embed the each chunk and upsert the embeddings into your pincone index.
docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [32]:
docsearch

In [33]:
retriver = docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [37]:
retrived_docs = retriver.invoke("What is the Bond")

In [38]:
retrived_docs

[Document(id='1957fd2f-aa6c-4fea-a514-1bff5b4d4047', metadata={'creationdate': '2022-03-16T14:06:07-05:00', 'creator': 'PyPDF', 'moddate': '2022-03-17T09:02:07-05:00', 'page': 326.0, 'page_label': '327', 'producer': 'Prince 14.2 (www.princexml.com)', 'source': 'Principle_of_finance.pdf', 'title': 'Principles of Finance', 'total_pages': 643.0}, page_content='Summary\n10.1 Characteristics of Bonds\nBonds are typically a basic form of investment that entails a straightforward financial agreement between\nissuer and purchaser. There are three primary categories of bonds: government bonds, corporate bonds, and\nconvertible bonds. These different types of bond vary depending on their issuer, length until maturity, interest\nrate, and risk.\n10.2 Bond Valuation'),
 Document(id='70247869-4640-4794-ab60-56d7db3250e6', metadata={'creationdate': '2022-03-16T14:06:07-05:00', 'creator': 'PyPDF', 'moddate': '2022-03-17T09:02:07-05:00', 'page': 326.0, 'page_label': '327', 'producer': 'Prince 14.2 (ww

In [46]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)


In [49]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are a helpful finance tutor. "
    "Use the following pieces of retrieved context to answer the student's question. "
    "If you don't know the answer, just say that you don't know. "
    "Use clear, simple language and keep your answer concise (maximum three sentences). "
    "If possible, provide a brief example or explanation."
    "\n\n"
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]

)


